<h1>Point_E Model Scanning Sample</h1>

<h3>Point_E Image To PointCloud<h3/>

In [1]:
from tqdm.auto import tqdm
from AIScan import CloudSampler
from AIScan.Models import CreateModel
from Utils.Data import getPathImages, getPathNPBinaries
from Utils.Torch import UseBestTorchDevice

from point_e.util.point_cloud import PointCloud
from point_e.util.mesh import TriMesh
from point_e.util.pc_to_mesh import marching_cubes_mesh
from point_e.util.plotting import plot_point_cloud

In [ ]:
srcPath = '../data/raw/sphere'
outputPath = '../data/Point_E/sphere'

In [2]:
# Loading Sampler For Chosen Cloud

sampler = CloudSampler.CreateCloudSampler('base1B')

Using cuda device for Torch Model.
Loading base40M Model
Loading Base Model Checkpoint


  0%|          | 0.00/162M [00:00<?, ?iB/s]

Loading upsample Model
Loading Base Model Checkpoint
Loading base40M Diffusion
Loading upsample Diffusion


In [3]:
# Loading Images From Provided Source Path (Provided In First Cell)

images = getPathImages(srcPath)
if (len(images) == 0):
    print("--__ No Images Found __--")
    exit()

27 images found in ../data/raw/sphere


In [4]:
# Generate Samples from the Sampler, Using Provided Images

samples = None

for x in tqdm(sampler.sample_batch_progressive(batch_size=len(images), model_kwargs=dict(images=images))):
    samples = x

del images

0it [00:00, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.64 GiB (GPU 0; 6.00 GiB total capacity; 3.37 GiB already allocated; 650.00 MiB free; 3.40 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# Save All Generated Samples To Cloud Files, 

figs = []

for i, cloud in enumerate(sampler.output_to_point_clouds(samples)):
    figs[i] = plot_point_cloud(cloud, grid_size=3, fixed_bounds=((-0.75, -0.75, -0.75),(0.75, 0.75, 0.75)))

    with open(f'{outputPath}/frags/cloud_{i}.ply', 'wb') as f:
        cloud.write_ply(f)

del sampler
del samples

In [ ]:
# Clear All Variables from Memory
del sampler
del samples

del figs

<h3>Point_E Pointcloud To Mesh</h3>

In [ ]:
# Create SDF Generator Model

sdfModel = CreateModel('sdf', UseBestTorchDevice())

In [ ]:
# Genreating Meshes From Clouds, Previously Saved To NPZ Files

curCloud: PointCloud = None
curMesh = None
meshes = []

for cloudPath in getPathNPBinaries(srcPath):
    curCloud = PointCloud.load(cloudPath)

    curMesh = marching_cubes_mesh(
        pc = curCloud,
        model = sdfModel,
        batch_size = 4096,
        grid_size = 128,
        progress=True
    )

    meshes.append(curMesh)